In [64]:
# Importing packages
import numpy as np
from numba import jit, njit
from pychange.segment import create_summary_stats
from pychange.numba_costs import normal_mean_var_cost

In [65]:
# Creating synthetic data
x = np.concatenate([np.random.normal(0, 1, (100,)),
                    np.random.normal(20, 2, (100,)),
                    np.random.normal(-10, 1, (100,)),
                   np.random.normal(10, 1, (100,))])

In [71]:
def pelt(x, min_len, penalty, cost_fn):
    """Pruned exact linear time changepoint segmentation"""
    
    # Setting up summary statistics and objects
    n = int(x.shape[0])
    sum_stats = np.stack((np.append(0, x.cumsum()),
                          np.append(0, (x ** 2).cumsum()),
                          np.append(0, ((x - x.mean()) ** 2).cumsum()),
                          np.arange(0, n + 1)),
                         axis=-1)
    
    # Initializing pelt parameters
    f = np.empty(shape=(n,), dtype=np.float64)
    f[0] = -penalty
    costs = np.empty(shape=(n,), dtype=np.float64)
    cp = [[]]
    r = np.array([0])
    
    
    # Entering main loop
    for tau_star in np.arange(1, n):
        
        # Calculating minimum segment cost
        costs[r] = cost_fn(sum_stats[tau_star] - sum_stats[r])
        _costs = costs[r] + f[r] + penalty
        
        f[tau_star] = _costs.min()
        tau_l = r[np.argmin(_costs)]
        
        # Setting new changepoints
        cp.append(cp[tau_l] + [tau_l])
        
        # Setting new candidate points
        r = np.append(r[(f[r] + costs[r] + penalty) <= f[tau_star]], tau_star)
        
    return cp[-1]

In [69]:
%timeit pelt(x, 10, 100, normal_mean_var_cost)

<ipython-input-68-9d5efe2fa6be>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "pelt" failed type inference due to: non-precise type pyobject
During: typing of argument at <ipython-input-68-9d5efe2fa6be> (6)

File "<ipython-input-68-9d5efe2fa6be>", line 6:
def pelt(x, min_len, penalty, cost_fn):
    <source elided>
    # Setting up summary statistics and objects
    n = int(x.shape[0])
    ^

  @jit(fastmath=True)
<ipython-input-68-9d5efe2fa6be>:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "pelt" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-68-9d5efe2fa6be>", line 22:
def pelt(x, min_len, penalty, cost_fn):
    <source elided>
    # Entering main loop
    for tau_star in np.arange(1, n):
    ^

  @jit(fastmath=True)
C:\Users\gursk\anaconda3\envs\pychange\lib\site-packages\numba\core\object_m

15.8 ms ± 127 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [70]:
%timeit pelt(x, 10, 100, normal_mean_var_cost)

16.1 ms ± 431 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [72]:
%timeit pelt(x, 10, 100, normal_mean_var_cost)

15.8 ms ± 689 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
